In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np



dataframe = pd.read_excel('triage dataset-coded 2.xlsx')

In [ ]:
print(dataframe)

In [ ]:
dataframe.shape

In [ ]:
dataframe = dataframe.dropna()

In [ ]:
dataframe.head()

In [ ]:
dataframe.shape

In [ ]:
selection_symptoms = dataframe.loc[:, ['gender','age', 'job','respcontact','placevisit','healthworker','fever','cough','sorethroat','vomdiarrhea', 'malaise','smelltaste','smoking','pregnancy','immundiff','comorbid',
    'dm',
    'htn',
    'cardiac',
    'ckd',
    'chest',
    'hepatic',
    'cancer',
    'others']]
selection_symptoms.shape

In [ ]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

selection_symptoms_train_dataframe = train_dataframe.loc[:, ['gender','age', 'job','respcontact','placevisit','healthworker','fever','cough','sorethroat','vomdiarrhea', 'malaise','smelltaste','smoking','pregnancy','immundiff','comorbid',
    'dm',
    'htn',
    'cardiac',
    'ckd',
    'chest',
    'hepatic',
    'cancer',
    'others','pcr']]
print(selection_symptoms_train_dataframe.shape)

selection_lab_train_dataframe = train_dataframe.loc[:, ['wbc',
    'neucount',
    'neu',
    'lymcount',
    'lym',
    'monocount',
    'mono',
    'eoscount',
    'eos',
    'basocount',
    'baso',
    'rbc',
    'hgb',
    'hct',
    'mcv',
    'mch',
    'mchc',
    'rdw',
    'plt',
    'mpv',
    'neutolym',
    'ast',
    'alt',
    'urea',
    'creat',
    'ldh',
    'ck',
    'crp',
    'ferritin',
    'inr','pcr']]

print(selection_lab_train_dataframe.shape)

# val

selection_symptoms_val_dataframe = val_dataframe.loc[:, ['gender','age', 'job','respcontact','placevisit','healthworker','fever','cough','sorethroat','vomdiarrhea', 'malaise','smelltaste','smoking','pregnancy','immundiff','comorbid',
    'dm',
    'htn',
    'cardiac',
    'ckd',
    'chest',
    'hepatic',
    'cancer',
    'others','pcr']]
print(selection_symptoms_val_dataframe.shape)

selection_lab_val_dataframe = val_dataframe.loc[:, ['wbc',
    'neucount',
    'neu',
    'lymcount',
    'lym',
    'monocount',
    'mono',
    'eoscount',
    'eos',
    'basocount',
    'baso',
    'rbc',
    'hgb',
    'hct',
    'mcv',
    'mch',
    'mchc',
    'rdw',
    'plt',
    'mpv',
    'neutolym',
    'ast',
    'alt',
    'urea',
    'creat',
    'ldh',
    'ck',
    'crp',
    'ferritin',
    'inr','pcr']]

print(selection_lab_val_dataframe.shape)



print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("pcr")
    train_X=dataframe.values.tolist()
    labels = labels.values.tolist()
    return train_X,labels


train_ds_1, y_1 = dataframe_to_dataset(selection_symptoms_train_dataframe)
train_ds_2, y_2 = dataframe_to_dataset(selection_lab_train_dataframe)



val_ds_1, y_1_v = dataframe_to_dataset(selection_symptoms_val_dataframe)
val_ds_2, y_2_v = dataframe_to_dataset(selection_lab_val_dataframe)

In [ ]:
train_ds_1, y_1 = np.array(train_ds_1), np.array(y_1)
train_ds_2, y_2 = np.array(train_ds_2), np.array(y_2)


val_ds_1, y_1_v = np.array(val_ds_1), np.array(y_1_v)
val_ds_2, y_2_v = np.array(val_ds_2), np.array(y_2_v)

In [ ]:
train_ds_1

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding
from tensorflow.keras.layers.experimental.preprocessing import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_string_categorical_feature(feature, name, dataset):
    # Create a StringLookup layer which will turn strings into integer indices
    index = StringLookup()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    index.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = index(feature)

    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a dataset of indices
    feature_ds = feature_ds.map(index)

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(encoded_feature)
    return encoded_feature


def encode_integer_categorical_feature(feature, name, dataset):
    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(feature)
    return encoded_feature

In [ ]:
train_ds_2.shape[-1]

In [ ]:
train_ds_1.shape[-1]

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32


all_symptoms_inputs = keras.layers.Input(shape = train_ds_1.shape[-1])
all_laboratory_inputs = keras.layers.Input(shape = train_ds_2.shape[-1])

dense_1 = keras.layers.Dense(32)(all_symptoms_inputs)
bn_1 = layers.BatchNormalization()(dense_1)
lrelu_1 = layers.LeakyReLU()(bn_1)
drop_1 = keras.layers.Dropout(0.5)(lrelu_1)
flat1 = keras.layers.Flatten()(drop_1)


dense_2 = keras.layers.Dense(32)(all_laboratory_inputs)
bn_2 = layers.BatchNormalization()(dense_2)
lrelu_2 = layers.LeakyReLU()(bn_2)
drop_2 = keras.layers.Dropout(0.5)(lrelu_2)
flat2 = keras.layers.Flatten()(drop_2)


# merge input models
x = keras.layers.Concatenate()([flat1, flat2])

x = keras.layers.Dense(16)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(8)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(1, activation="sigmoid")(x)

model = keras.Model([all_symptoms_inputs, all_laboratory_inputs], output)

model.compile(loss="binary_crossentropy", optimizer="Adam",metrics=["accuracy"])

In [ ]:
# `rankdir='LR'` is to make the graph horizontal.
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.summary()

In [ ]:
#from tensorflow.keras.callbacks import TensorBoard

#NAME = "triage-v3"
#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [ ]:
# checkpoint
filepath="triage_v4_dual_modal_weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit([train_ds_1, train_ds_2], y_1, validation_data=([val_ds_1, val_ds_2], y_1_v), epochs=100, batch_size=32, callbacks=callbacks_list)

In [ ]:
#model.save('triage-v4-dual-modal')


In [ ]:
all_symptoms_inputs